In [80]:
import streamlit as st
from pymongo import MongoClient
from datetime import datetime

# ---------- DATABASE CONNECTION ---------- #
def connect_to_mongodb():
    client = MongoClient("mongodb://localhost:27017/")
    return client["news_articles"]

# ---------- CATEGORY & ISSUE MAPPING ---------- #
category_issues = {
    "education": [
        "education inequality", "remote learning", "diversity education",
        "school funding", "student loan debt"
    ],
    "politics": [
        "foreign policy", "partisan divide", "elections",
        "immigration policy", "education policy"
    ],
    "environment": [
        "carbon emission", "climate change", "green tech",
        "recycling", "remote learning", "sustainability"
    ]
}

# ---------- HELPER FUNCTIONS ---------- #
def fetch_articles_mongo(db, collection_name, issue):
    return list(db[collection_name].find({"issue": issue}).sort("published_date", 1).limit(5))

def get_top_issue_by_category_mongo(db, collection_name, category):
    pipeline = [
        {"$match": {"issue": {"$in": category_issues[category]}}},
        {"$group": {"_id": "$issue", "count": {"$sum": 1}}},
        {"$sort": {"count": -1}},
        {"$limit": 1}
    ]
    result = list(db[collection_name].aggregate(pipeline))
    if result:
        return result[0]['_id'], result[0]['count']
    else:
        return "N/A", 0

def display_articles_mongo(collection_name, mongo_articles, total_cat_count, total_issue_count):
    display_name = "FOXNEWS" if collection_name == "newsapi" else collection_name.upper()

    st.markdown(f"""
    <div style='background-color:white; padding:15px; border-radius:12px; 
                border: 2px solid #333; margin-bottom: 20px; box-shadow: 2px 2px 8px rgba(0,0,0,0.2);'>
        <h3 style='color:#333;'> {display_name}</h3>
        <b>Total Articles in Category:</b> {total_cat_count}<br>
        <b>Total Articles in Issue:</b> {total_issue_count}<br>
        <hr>
        <b>First 5 Articles:</b><br>
    """, unsafe_allow_html=True)

    for article in mongo_articles:
        title = article.get("title", "No Title")
        author = article.get("author", "N/A")
        published_date = article.get("published_date", "N/A")
        url = article.get("web_url", article.get("url", "#"))

        st.markdown(f"""
        <div style='background-color:white; padding:10px; margin-bottom:10px; border-radius:8px;'>
        <b>Title:</b> {title}<br>
        <b>Author:</b> {author}<br>
        <b>Published Date:</b> {format_date(str(published_date))}<br>
        <b>URL:</b> <a href='{url}' target='_blank'>Link</a>
        </div>
        """, unsafe_allow_html=True)

    st.markdown("</div>", unsafe_allow_html=True)

# ---------- UTILITIES ---------- #
def format_date(date_str):
    if not date_str:
        return "N/A"
    try:
        dt = datetime.fromisoformat(date_str.replace("Z", "+00:00"))
        return dt.strftime("%Y-%m-%d %H:%M")
    except Exception:
        return date_str

# ---------- MAIN APP ---------- #

def run_news_dashboard():
    db = connect_to_mongodb()
    page_bg_img = f"""
    <style>
    [data-testid="stAppViewContainer"] > .main {{
        background-image: url("https://www.shutterstock.com/image-vector/newspaper-background-torn-paper-style-600nw-2261765635.jpg");
        background-size: cover;
        background-position: center;
        background-repeat: no-repeat;
        background-attachment: fixed;
    }}
    
    [data-testid="stHeader"] {{
        background: rgba(255, 255, 255, 0.0);
    }}
    
    .white-box {{
        background-color: rgba(245, 245, 220, 0.95);
        padding: 2rem;
        border-radius: 15px;
        box-shadow: 2px 2px 15px rgba(0,0,0,0.1);
        max-width: 1000px;
        margin: 3rem auto;
    }}
    
    h1 {{
        text-align: center;
        font-size: 3em;
        color: #1f2937;
        margin-top: 2rem;
    }}
    </style>
    """
    st.markdown(page_bg_img, unsafe_allow_html=True)

    st.markdown("""
    <style>
    .small-white-box {
    background-color: rgba(245, 245, 220, 0.95);
    padding: 1.5rem;
    border-radius: 15px;
    box-shadow: 2px 2px 10px rgba(0,0,0,0.1);
    max-width: 500px;
    margin: 2rem auto 1rem auto;
    }
    </style>
    """, unsafe_allow_html=True)
    
    
    with st.container():
        #st.markdown('<div class="small-white-box">', unsafe_allow_html=True)

        # Dropdowns inside the white box
        category = st.selectbox("Select a Category:", list(category_issues.keys()))
        issue = st.selectbox("Select an Issue:", category_issues[category])
    
        st.markdown('</div>', unsafe_allow_html=True)
    
        # Horizontal rule (divider)
        st.markdown("<hr>", unsafe_allow_html=True)
    
        # Heading for the issues section
        st.markdown("""
        <div style='background-color:white; padding:15px; border-radius:12px;
                    border: 2px solid #333; margin-bottom: 20px; box-shadow: 2px 2px 8px rgba(0,0,0,0.2);'>
            <h4 style='color:#333;'>Issues with the Maximum Articles across 3 Categories:</h4>
        </div>
        """, unsafe_allow_html=True)


    # Build the info string
    max_issue_info = ""
    for source in ["nytimes", "newsapi", "guardian"]:
        top_issue, count = get_top_issue_by_category_mongo(db, source, category)
        display_name = "FOXNEWS" if source == "newsapi" else source.upper()
        max_issue_info += f"<p><b>{display_name}:</b> {top_issue} ({count} articles)</p>"

    # Display it inside a white box
    st.markdown(f"""
    <div class="small-white-box">
    <h3 style='text-align:center;'>Top Issues by Source</h3>
    {max_issue_info}
    </div>
    """, unsafe_allow_html=True)

   # st.markdown(max_issue_info, unsafe_allow_html=True)
  #  st.markdown("</div>", unsafe_allow_html=True)
  #  st.markdown("<hr>", unsafe_allow_html=True)

    col1, col2, col3 = st.columns(3)

    for col, source in zip([col1, col2, col3], ["nytimes", "newsapi", "guardian"]):
        with col:
            mongo_articles = fetch_articles_mongo(db, source, issue)
            total_cat = len(list(db[source].find({"issue": {"$in": category_issues[category]}})))
            total_issue = len(mongo_articles)
            display_articles_mongo(source, mongo_articles, total_cat, total_issue)

# ---------- STREAMLIT ENTRY POINT ---------- #
if __name__ == '__main__':
    run_news_dashboard()